## Imports all libraries

In [1]:
import requests
from urllib.request import urlopen

import json

import numpy as np
import pandas as pd

import plotly.express as px
import plotly.io as pio

from datetime import date, datetime

## Request the total cases in germany for every day.

In [2]:
## Get data from the api

reqUrl = "https://api.corona-zahlen.org/germany/history/cases"

headersList = {
 "Accept": "*/*" 
}

payload = ""

response = requests.request("GET", reqUrl, data=payload,  headers=headersList)

data_cases = response.json()

#### Prepare the structure

In [3]:
# Convert data to a dataframe

cases = pd.json_normalize(data_cases['data'])

cases.head(10)

,cases,date
0,0,2020-01-01T00:00:00.000Z
1,1,2020-01-02T00:00:00.000Z
2,0,2020-01-03T00:00:00.000Z
3,0,2020-01-04T00:00:00.000Z
4,0,2020-01-05T00:00:00.000Z
5,0,2020-01-06T00:00:00.000Z
6,0,2020-01-07T00:00:00.000Z
7,0,2020-01-08T00:00:00.000Z
8,0,2020-01-09T00:00:00.000Z
9,0,2020-01-10T00:00:00.000Z


#### Start working with your data

In [ ]:
## Check the formats

cases.dtypes

In [ ]:
## Convert date to date format

cases["date"] = pd.to_datetime(cases["date"])

cases.dtypes

In [6]:
## Sort data by date

cases.sort_values(by='date', ascending = True, inplace=True)

cases.head(10)

,cases,date
0,0,2020-01-01 00:00:00+00:00
1,1,2020-01-02 00:00:00+00:00
2,0,2020-01-03 00:00:00+00:00
3,0,2020-01-04 00:00:00+00:00
4,0,2020-01-05 00:00:00+00:00
5,0,2020-01-06 00:00:00+00:00
6,0,2020-01-07 00:00:00+00:00
7,0,2020-01-08 00:00:00+00:00
8,0,2020-01-09 00:00:00+00:00
9,0,2020-01-10 00:00:00+00:00


In [7]:
## Sort data by most cases

cases.sort_values(by='cases', ascending = False, inplace=True)

cases.head(10)

,cases,date
812,307914,2022-03-23 00:00:00+00:00
805,307368,2022-03-16 00:00:00+00:00
811,304773,2022-03-22 00:00:00+00:00
806,294874,2022-03-17 00:00:00+00:00
813,286333,2022-03-24 00:00:00+00:00
804,285044,2022-03-15 00:00:00+00:00
818,274406,2022-03-29 00:00:00+00:00
798,261299,2022-03-09 00:00:00+00:00
819,260011,2022-03-30 00:00:00+00:00
807,254602,2022-03-18 00:00:00+00:00


In [8]:
## Filter out current year's values

year = datetime.today().strftime('%Y')

cases_cy = cases[cases['date'].dt.year == int(year)]

cases_cy.head(10)

,cases,date
812,307914,2022-03-23 00:00:00+00:00
805,307368,2022-03-16 00:00:00+00:00
811,304773,2022-03-22 00:00:00+00:00
806,294874,2022-03-17 00:00:00+00:00
813,286333,2022-03-24 00:00:00+00:00
804,285044,2022-03-15 00:00:00+00:00
818,274406,2022-03-29 00:00:00+00:00
798,261299,2022-03-09 00:00:00+00:00
819,260011,2022-03-30 00:00:00+00:00
807,254602,2022-03-18 00:00:00+00:00


In [9]:
## Group cases by months

cases_cy_by_month = cases_cy.groupby(cases_cy['date'].dt.strftime('%m %B'))['cases'].sum().sort_index()

cases_cy_by_month

date
01 January      2920929
02 February     4858242
03 March        6558685
04 April        3400199
05 May          1475122
06 June         1955312
07 July         2558950
08 August       1282970
09 September    1216138
10 October      1750989
Name: cases, dtype: int64

#### Visualize your data

In [ ]:
## Display cases over time

## If you have issues displaying the chart in your Data Platform Appliance, uncomment the following line and restart the kernel. Rerun everything from the beginning.
# pio.renderers.default = 'iframe'

cases['month'] = cases['date'].dt.month

fig = px.bar(cases, x='date', y="cases", title="Cases in Germany over time")

fig.show()

### Request corona stats per state

In [11]:
## Get data from the api

reqUrl = "https://api.corona-zahlen.org/states"

headersList = {
 "Accept": "*/*" 
}

payload = ""

response = requests.request("GET", reqUrl, data=payload,  headers=headersList)

data_cases_by_state = response.json()

#### Prepare the structure

In [12]:
## Convert data to a dataframe

cases_by_state = pd.DataFrame.from_dict(data_cases_by_state['data'], orient='index')

for index, row in cases_by_state.iterrows():
    delta = row["delta"]
    if not pd.isnull(delta):
        for k, v in delta.items():
            key = f'delta_{k}'
            if key not in cases_by_state:
                cases_by_state[key] = np.nan
            cases_by_state.loc[index, key] = v
cases_by_state.drop(columns=["delta"], inplace=True)

for index, row in cases_by_state.iterrows():
    hospitalization = row["hospitalization"]
    if not pd.isnull(hospitalization):
        for k, v in hospitalization.items():
            key = f'hospitalization_{k}'
            if key not in cases_by_state:
                cases_by_state[key] = np.nan
            cases_by_state.loc[index, key] = v
cases_by_state.drop(columns=["hospitalization"], inplace=True)

cases_by_state.head(10)

,id,name,population,cases,deaths,casesPerWeek,deathsPerWeek,recovered,abbreviation,weekIncidence,casesPer100k,delta_cases,delta_deaths,delta_recovered,delta_weekIncidence,hospitalization_cases7Days,hospitalization_incidence7Days,hospitalization_date,hospitalization_lastUpdate
SH,1,Schleswig-Holstein,2922005,1071070,2987,14925,5,1011825,SH,510.779413,36655.310309,2245.0,7.0,637.0,-11.362061,422.0,14.44,2022-10-22T00:00:00.000Z,2022-10-22T03:05:27.000Z
HH,2,Hamburg,1853935,750179,3113,5967,0,734129,HH,321.855944,40464.147880,1430.0,7.0,318.0,18.878763,99.0,5.34,2022-10-22T00:00:00.000Z,2022-10-22T03:05:27.000Z
NI,3,Niedersachsen,8027031,3415236,11110,54868,13,3248870,NI,683.540402,42546.690053,7786.0,30.0,2752.0,-20.829619,537.0,6.69,2022-10-22T00:00:00.000Z,2022-10-22T03:05:27.000Z
HB,4,Bremen,676463,273397,854,3918,1,263043,HB,579.189106,40415.662054,624.0,1.0,138.0,12.269703,35.0,5.17,2022-10-22T00:00:00.000Z,2022-10-22T03:05:27.000Z
NW,5,Nordrhein-Westfalen,17924591,7179653,27946,116296,30,6854114,NW,648.806994,40054.766103,18241.0,38.0,6847.0,-2.750411,2346.0,13.09,2022-10-22T00:00:00.000Z,2022-10-22T03:05:27.000Z
HE,6,Hessen,6295017,2689427,11297,48828,14,2493231,HE,775.661130,42723.109405,6006.0,30.0,1954.0,-52.994297,798.0,12.68,2022-10-22T00:00:00.000Z,2022-10-22T03:05:27.000Z
RP,7,Rheinland-Pfalz,4106485,1628198,6320,31213,10,1498051,RP,760.090442,39649.432544,5524.0,13.0,431.0,8.060422,475.0,11.57,2022-10-22T00:00:00.000Z,2022-10-22T03:05:27.000Z
BW,8,Baden-Württemberg,11124642,4761962,17582,66633,23,4484570,BW,598.967589,42805.530281,9271.0,16.0,1794.0,-42.760927,1128.0,10.14,2022-10-22T00:00:00.000Z,2022-10-22T03:05:27.000Z
BY,9,Bayern,13176989,6398266,26273,78785,26,6064629,BY,597.898351,48556.358361,9147.0,31.0,8777.0,-37.709677,2111.0,16.02,2022-10-22T00:00:00.000Z,2022-10-22T03:05:27.000Z
SL,10,Saarland,982348,453145,1897,9837,5,420283,SL,1001.376294,46128.764959,1146.0,1.0,-5.0,-56.700884,291.0,29.62,2022-10-22T00:00:00.000Z,2022-10-22T03:05:27.000Z


#### Start working with your data

In [ ]:
## Check the formats

cases_by_state.dtypes

In [ ]:
## Convert date to date format

cases_by_state["hospitalization_lastUpdate"] = pd.to_datetime(cases_by_state["hospitalization_lastUpdate"])
cases_by_state["hospitalization_date"] = pd.to_datetime(cases_by_state["hospitalization_date"])

cases_by_state.dtypes

#### Visualize your data

In [ ]:
## Plot cases by state

with urlopen('https://raw.githubusercontent.com/isellsoap/deutschlandGeoJSON/main/2_bundeslaender/1_sehr_hoch.geo.json') as response:
    counties = json.load(response)

fig = px.choropleth(cases_by_state, geojson=counties, locations='name', featureidkey="properties.name", color='cases',
                           color_continuous_scale="OrRd",
                           projection="mercator",
                           range_color=(0, 5500000),
                           labels={'name':'State', 'cases':'Cases'}
                          )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(title_text = "Today's cases in Germany by state", margin={"r":0,"t":50,"l":0,"b":50})
fig.show()

In [ ]:
## Plot cases of the last 7 days by state

last_update = cases_by_state['hospitalization_lastUpdate'].iloc[1].strftime("%b %d %Y %H:%M")

fig = px.choropleth(cases_by_state, geojson=counties, locations='name', featureidkey="properties.name", color='hospitalization_cases7Days',
                           color_continuous_scale="OrRd",
                           projection="mercator",
                           range_color=(0, 1500),
                           labels={'name':'State', 'hospitalization_cases7Days':'Cases7Days'}
                          )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(title_text = f"Hospitalization cases of the last 7 days in Germany by state (Last Updated: {last_update})", margin={"r":0,"t":50,"l":0,"b":50})
fig.show()

In [ ]:
## Analyze correlation between variables

fig = px.scatter(cases_by_state, x="cases", y="population", trendline="ols", hover_name='name', title="Correlation between population size and number of cases")
fig.show()

### Request hospitalization stats per state

In [18]:
## Get data from the api

reqUrl = "https://api.corona-zahlen.org/states/history/hospitalization"

headersList = {
 "Accept": "*/*" 
}

payload = ""

response = requests.request("GET", reqUrl, data=payload,  headers=headersList)

data_hospitalization = response.json()

#### Prepare the structure

In [19]:
## Convert data to a dataframe

df_hospitalization = pd.DataFrame.from_dict(data_hospitalization['data'], orient='index')

hospitalization_by_state = (pd.concat({i: pd.json_normalize(x) for i, x in df_hospitalization.pop('history').items()})
         .reset_index(level=1, drop=True)
         .join(df_hospitalization))
         
hospitalization_by_state.head(10)

,cases7Days,incidence7Days,date,fixedCases7Days,updatedCases7Days,adjustedLowerCases7Days,adjustedCases7Days,adjustedUpperCases7Days,fixedIncidence7Days,updatedIncidence7Days,adjustedLowerIncidence7Days,adjustedIncidence7Days,adjustedUpperIncidence7Days,id,name
BB,0,0.00,2020-03-01T00:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,Brandenburg
BB,0,0.00,2020-03-02T00:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,Brandenburg
BB,0,0.00,2020-03-03T00:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,Brandenburg
BB,0,0.00,2020-03-04T00:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,Brandenburg
BB,0,0.00,2020-03-05T00:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,Brandenburg
BB,0,0.00,2020-03-06T00:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,Brandenburg
BB,0,0.00,2020-03-07T00:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,Brandenburg
BB,0,0.00,2020-03-08T00:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,Brandenburg
BB,1,0.04,2020-03-09T00:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,Brandenburg
BB,2,0.08,2020-03-10T00:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,Brandenburg


#### Visualize your data

In [ ]:
## Plot cases7Days by state and date

fig = px.line(hospitalization_by_state, x='date', y='cases7Days', color=hospitalization_by_state.index, title='Hospitalization cases for last 7 days per state', hover_name='name')

fig.update_xaxes(rangeslider_visible=True)
fig.show()

### Request recovered stats per state

In [21]:
## Get data from the api

reqUrl = "https://api.corona-zahlen.org/states/history/recovered"

headersList = {
 "Accept": "*/*" 
}

payload = ""

response = requests.request("GET", reqUrl, data=payload,  headers=headersList)

data_recovered = response.json()

#### Prepare the structure

In [22]:
## Convert data to a dataframe

df_recovered = pd.DataFrame.from_dict(data_recovered['data'], orient='index')

recovered_by_state = (pd.concat({i: pd.json_normalize(x) for i, x in df_recovered.pop('history').items()})
         .reset_index(level=1, drop=True)
         .join(df_recovered))

recovered_by_state.head(10)

,recovered,date,id,name
BB,0,2020-01-01T00:00:00.000Z,12,Brandenburg
BB,0,2020-01-02T00:00:00.000Z,12,Brandenburg
BB,0,2020-01-03T00:00:00.000Z,12,Brandenburg
BB,0,2020-01-04T00:00:00.000Z,12,Brandenburg
BB,0,2020-01-05T00:00:00.000Z,12,Brandenburg
BB,0,2020-01-06T00:00:00.000Z,12,Brandenburg
BB,0,2020-01-07T00:00:00.000Z,12,Brandenburg
BB,0,2020-01-08T00:00:00.000Z,12,Brandenburg
BB,0,2020-01-09T00:00:00.000Z,12,Brandenburg
BB,0,2020-01-10T00:00:00.000Z,12,Brandenburg


#### Visualize your data

In [ ]:
## Plot recovered by state and date

fig = px.line(recovered_by_state, x='date', y='recovered', color=recovered_by_state.index, title='Recovered per state', hover_name='name')
fig.update_xaxes(rangeslider_visible=True)
fig.show()